In [6]:
import py2neo
print(py2neo.__version__)

4.0.0b2


In [7]:
# python libraries
from py2neo import Graph
import numpy as np 
from pandas import DataFrame
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import json
import math
import pandas as pd
# import plotly 
# import plotly.graph_objs as go
# import qgrid
from scipy import stats, spatial
from scipy.sparse import csr_matrix
from sklearn.cluster.bicluster import SpectralBiclustering
import scipy.spatial as sp, scipy.cluster.hierarchy as hc, scipy.sparse.csgraph as csgraph
import operator
from IPython.display import display, HTML
from matplotlib.colors import ListedColormap


# connection to Neo4j
local_connection_url = "http://localhost:7474/db/data"
connection_to_graph = Graph(local_connection_url, user="neo4j", password="zxc397") 

# plotly credentials
# plotly_config = json.load(open('plotly_config.json'))
# plotly.tools.set_credentials_file(username=plotly_config['username'], api_key=plotly_config['key'])

In [145]:
f_terms = list(set(DataFrame(connection_to_graph.data('MATCH (a:Asset)-[:CONTAINS]->(fs:Feedstock) RETURN  fs.term, count(a)')).as_matrix()[:, 1]))
o_terms = list(set(DataFrame(connection_to_graph.data('MATCH (a:Asset)-[:CONTAINS]->(fs:Output) RETURN  fs.term, count(a)')).as_matrix()[:, 1]))
pt_terms = list(set(DataFrame(connection_to_graph.data('MATCH (a:Asset)-[:CONTAINS]->(fs:ProcessingTech) RETURN  fs.term, count(a)')).as_matrix()[:, 1]))
bbo = list(f_terms + pt_terms + o_terms)
print ("Number of terms:", len(bbo))
nodes = bbo
print(nodes)

('Number of terms:', 352)
[u'vegetable oil', u'recycled ethanol', u'woodwaste/bagasse', u'natural gas', u'various grasses', u'nonedible oils', u'animal fats', u'paper', u'ponderosa pine', u'grass seed', u'sulfite spent liquor from spruce wood pulping', u'osb', u'paper waste', u'corn cob', u'energy grasses', u'logging residues', u'willow', u'crop waste', u'firewood', u'jatropha', u'sugarcane bagasse', u'sugar', u'durum', u'wood', u'municipal solid waste', u'banna grass', u'waste oil', u'cassava pulp', u'barley', u'mixed oilseeds', u'starch', u'white grease', u'coniferous wood', u'corn oil', u'wood waste', u'mixed cellulose', u'corn stalks', u'yellow grease', u'molasses', u'woodchips', u'food waste', u'citrus residues', u'wood chips', u'spent sulphite liquor feedstock', u'palm, rapeseed oil, waste fat', u'cotton residue', u'industrial waste', u'deciduous forests', u'corn stover', u'cereals/sugar', u'waste fat', u'waste vegetable oil', u'tallow', u'sewage', u'white pine', u'hardwood', u'b

In [146]:
#Remove leading spaces
newnodes = list()
for node in nodes:
    newnode = node.lstrip()
    newnodes.append(newnode)
nodes=newnodes
nodes

[u'vegetable oil',
 u'recycled ethanol',
 u'woodwaste/bagasse',
 u'natural gas',
 u'various grasses',
 u'nonedible oils',
 u'animal fats',
 u'paper',
 u'ponderosa pine',
 u'grass seed',
 u'sulfite spent liquor from spruce wood pulping',
 u'osb',
 u'paper waste',
 u'corn cob',
 u'energy grasses',
 u'logging residues',
 u'willow',
 u'crop waste',
 u'firewood',
 u'jatropha',
 u'sugarcane bagasse',
 u'sugar',
 u'durum',
 u'wood',
 u'municipal solid waste',
 u'banna grass',
 u'waste oil',
 u'cassava pulp',
 u'barley',
 u'mixed oilseeds',
 u'starch',
 u'white grease',
 u'coniferous wood',
 u'corn oil',
 u'wood waste',
 u'mixed cellulose',
 u'corn stalks',
 u'yellow grease',
 u'molasses',
 u'woodchips',
 u'food waste',
 u'citrus residues',
 u'wood chips',
 u'spent sulphite liquor feedstock',
 u'palm, rapeseed oil, waste fat',
 u'cotton residue',
 u'industrial waste',
 u'deciduous forests',
 u'corn stover',
 u'cereals/sugar',
 u'waste fat',
 u'waste vegetable oil',
 u'tallow',
 u'sewage',
 u'w

In [147]:
#REMOVAL/ADDITION - find nodes with commas, separate them and add to nodelist if not in there, remove the unseparated node
print len(nodes)
for node in nodes:
    if "," in node:
        separates = node.split(",")
        for separate in separates:
            #remove leading space if any
            separate = separate.lstrip()
            if separate not in nodes:
                nodes.append(separate)
                print separate
        #remove unseparated node from list
        nodes.remove(node)
print len(nodes)
print nodes

352
methanol synthesis
mtg catalysis
fibers
polylutylene succinate
bioplymers from corn
potatoes
tapioca
renewable diesel from animal fats
veggie oils
352
[u'vegetable oil', u'recycled ethanol', u'woodwaste/bagasse', u'natural gas', u'various grasses', u'nonedible oils', u'animal fats', u'paper', u'ponderosa pine', u'grass seed', u'sulfite spent liquor from spruce wood pulping', u'osb', u'paper waste', u'corn cob', u'energy grasses', u'logging residues', u'willow', u'crop waste', u'firewood', u'jatropha', u'sugarcane bagasse', u'sugar', u'durum', u'wood', u'municipal solid waste', u'banna grass', u'waste oil', u'cassava pulp', u'barley', u'mixed oilseeds', u'starch', u'white grease', u'coniferous wood', u'corn oil', u'wood waste', u'mixed cellulose', u'corn stalks', u'yellow grease', u'molasses', u'woodchips', u'food waste', u'citrus residues', u'wood chips', u'spent sulphite liquor feedstock', u'cotton residue', u'industrial waste', u'deciduous forests', u'corn stover', u'cereals/suga

In [11]:
#REMOVAL - all edges with NULL Years
feedproc = connection_to_graph.data('MATCH p2=(feed:Feedstock)<-[:CONTAINS]-(n:Asset)-[:CONTAINS]->(proc:ProcessingTech) WHERE (n.year)<>"Null" RETURN n.id, n.year, feed.term, proc.term')
feedout = connection_to_graph.data('MATCH p2=(feed:Feedstock)<-[:CONTAINS]-(n:Asset)-[:CONTAINS]->(out:Output) WHERE (n.year)<>"Null" RETURN n.id, n.year, feed.term, out.term')
procout = connection_to_graph.data('MATCH p2=(proc:ProcessingTech)<-[:CONTAINS]-(n:Asset)-[:CONTAINS]->(out:Output) WHERE (n.year)<>"Null" RETURN n.id, n.year, proc.term, out.term')
df1 = pd.DataFrame(feedproc)
df2 = pd.DataFrame(feedout)
df3 = pd.DataFrame(procout)

In [12]:
edgelist = []
for index, row in df1.iterrows():
    rowstr = row[u'feed.term'].lstrip() + "," + row[u'proc.term'].lstrip() + "," + row[u'n.year']
    #edgelist = edgelist + str
    edgelist.append(rowstr)
# print "df2-------------"
for index, row in df2.iterrows():
    rowstr = row[u'feed.term'].lstrip() + "," + row[u'out.term'].lstrip() + "," + row[u'n.year']
    edgelist.append(rowstr)
    #edgelist = edgelist + str
# print "df3-------------"
for index, row in df3.iterrows():
    rowstr = row[u'proc.term'].lstrip() + "," + row[u'out.term'].lstrip() + "," + row[u'n.year']
    edgelist.append(rowstr)
    #edgelist = edgelist + str
edgelist

[u'cooking oil,solvents,2011',
 u'waste,solvents,2011',
 u'waste,solvents,2012',
 u'industrial waste,solvents,2012',
 u'free fatty acid,solvents,2012',
 u'corn stover,solvents,2016',
 u'stover,solvents,2016',
 u'sugar,solvents,2016',
 u'corn,solvents,2016',
 u'cellulose,solvents,2016',
 u'cellulosic biomass,solvents,2016',
 u'cellulose,solvents,2012',
 u'douglas fir,solvents,2014',
 u'wood,solvents,2014',
 u'woody biomass,solvents,2014',
 u'cellulose,solvents,2014',
 u'pine,solvents,2013',
 u'wood,solvents,2013',
 u'sawdust,solvents,2013',
 u'waste water,solvents,2014',
 u'algae,solvents,2014',
 u'waste,solvents,2014',
 u'waste water,solvents,2016',
 u'palm,solvents,2016',
 u'waste,solvents,2016',
 u'sewage,solvents,2016',
 u'vegetable oil,solvents,2017',
 u'cooking oil,solvents,2017',
 u'waste,solvents,2017',
 u'waste,solvents,2016',
 u'waste water,solvents,2011',
 u'algae,solvents,2011',
 u'pal waste,solvents,2011',
 u'waste,solvents,2011',
 u'sewage,solvents,2011',
 u'algae,solvents

In [13]:
import csv

with open('edgelist', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(edgelist)

In [14]:
df = pd.DataFrame({'edges':edgelist}) #convert list to dataframe

In [15]:
# convert into weighted edgelist - count occurences for each edge
df['source'], df['target'], df['year'] = df['edges'].str.split(',', 2).str
weighted = df.groupby(['edges','source','target','year']).size().reset_index(name='count')
#weightedless = df.groupby(['edges']).size().reset_index(name='count')
#weightedless
weighted

,edges,source,target,year,count
0,"Giant reed,advanced biofuel,2013",Giant reed,advanced biofuel,2013,1
1,"Giant reed,anaerobic digestion,2014",Giant reed,anaerobic digestion,2014,2
2,"Giant reed,anaerobic digestion,2015",Giant reed,anaerobic digestion,2015,1
3,"Giant reed,anaerobic digestion,2016",Giant reed,anaerobic digestion,2016,2
4,"Giant reed,biodiesel,2011",Giant reed,biodiesel,2011,2
5,"Giant reed,biodiesel,2012",Giant reed,biodiesel,2012,1
6,"Giant reed,biogas,2012",Giant reed,biogas,2012,1
7,"Giant reed,biogas,2014",Giant reed,biogas,2014,2
8,"Giant reed,biogas,2016",Giant reed,biogas,2016,2
9,"Giant reed,butanol,2012",Giant reed,butanol,2012,1


In [148]:
#TODO - REWRITE THIS AS FOR ONE YEAR
#REMOVAL - remove nodes from nodelist that are not in edges
print len(nodes)
toremove = []
for node in nodes:
    i = 0
    #check if node is in source
    for source in df['source']:
        if node == source:
            i = 1
    #check if node is in source
    for target in df['target']:
        if node == target:
            i = 1
    if i == 0:
        toremove.append(node)
        nodes.remove(node)
print (toremove)
print (len(nodes))

352
[u'recycled ethanol', u'sulfite spent liquor from spruce wood pulping', u'durum', u'cassava pulp', u'mixed oilseeds', u'white grease', u'mixed cellulose', u'spent sulphite liquor feedstock', u'deciduous forests', u'cereals/sugar', u'biogas from municipal wastewater treatment facility digesters', u'stump material', u'animal residue', u'fall rye', u'fat products', u'soft white spring wheat', u'mixed grass', u'poppy seed', u'multi feedstock', u'palm stearin', u'mixedwood', u'corn sugar', u'feed wheat', u'juncea oil', u'barley starch', u'Decommissioned electricity poles and railway ties', u'cps wheat', u'radiate pine', u'red pine', u'biogenic waste oils', u'bagasse/cane trash', u'wood processing residues', u'natural fats', u'steel waste gas (co)', u'flaxseed oil', u'corn/milo', u'non-food grade canola oil', u'algae transesterification', u'solar conversion', u'chemprocessing', u'orc process', u'btl - syngas', u'algae burning/transesterifcation', u'steam reform/ft', u'algae oil hydrotrea

In [149]:
#nodeindex = {0:'zero',1:'one'}
#create dictionary to map indexes to nodes
nodeindex = {}
for i in range(len(nodes)):
    nodeindex[i] = nodes[i]
nodeindex

{0: u'vegetable oil',
 1: u'woodwaste/bagasse',
 2: u'natural gas',
 3: u'various grasses',
 4: u'nonedible oils',
 5: u'animal fats',
 6: u'paper',
 7: u'ponderosa pine',
 8: u'grass seed',
 9: u'osb',
 10: u'paper waste',
 11: u'corn cob',
 12: u'energy grasses',
 13: u'logging residues',
 14: u'willow',
 15: u'crop waste',
 16: u'firewood',
 17: u'jatropha',
 18: u'sugarcane bagasse',
 19: u'sugar',
 20: u'wood',
 21: u'municipal solid waste',
 22: u'banna grass',
 23: u'waste oil',
 24: u'barley',
 25: u'starch',
 26: u'coniferous wood',
 27: u'corn oil',
 28: u'wood waste',
 29: u'corn stalks',
 30: u'yellow grease',
 31: u'molasses',
 32: u'woodchips',
 33: u'food waste',
 34: u'citrus residues',
 35: u'wood chips',
 36: u'cotton residue',
 37: u'industrial waste',
 38: u'corn stover',
 39: u'waste fat',
 40: u'waste vegetable oil',
 41: u'tallow',
 42: u'sewage',
 43: u'white pine',
 44: u'hardwood',
 45: u'loblolly pine',
 46: u'miscanthus',
 47: u'palm',
 48: u'milo',
 49: u'b

In [18]:
# create matrix with zeros with size nodes x nodes
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
zero_data = np.zeros(shape=(len(nodes),len(nodes)))
adjacency = pd.DataFrame(zero_data, index = nodeindex, columns=nodes)
adjacency.rename(index=nodeindex, inplace=True)
#d.rename(nodes, axis='index')
#dists.at[' aerobic digestion',' aerobic digestion'] = 1

In [19]:
if "anaerobic digestion" in nodes:
    print "yes"
else:
    print "no"

yes


In [20]:
#fill adjacency matrix with actual distances, i.e. edge weights
#for each row in edgelist df 
    #put count into source/target cell in distmatrix / d.at[nodes.index(source),target] = count
for index, row in weighted.iterrows():
    #if nodes.isin(row.source) and nodes.isin(row.target) :
    #print row['count']
    if row.source in nodes and row.target in nodes:
        adjacency.at[row.source,row.target] = row['count']
        adjacency.at[row.target,row.source] = row['count']
    else:
        print (row.source + ":" + row.target)
%store adjacency
%store nodes

corn stover: cobs
corn stover: cobs
corn stover: cobs
corn stover: cobs
rice straw: corn stalks
rice straw: corn stalks
rice straw: corn stalks
Stored 'adjacency' (DataFrame)
Stored 'nodes' (list)


In [21]:
#restore clustering from iGraph
#x['a'] = 1
%store -r clusts
clusts_nodes = dict()
for i, cluster in enumerate(clusts):
#     print ("Cluster: " + str(i))
#     print ("Size: " + str(len(cluster)))
    cnodes = list()
    for item in cluster:
        #print(nodes[item])
        #add node names to cluster
        cnodes.append(nodes[item])
    #print ("\n")
    clusts_nodes[i]=cnodes
#print# clusts_nodes

In [22]:
#swap keys with values
nodesdict = dict()
for key, values in clusts_nodes.iteritems():
    for value in values:
        nodesdict[value] = key
#nodesdict

In [23]:
col_names =  ['year', 'node', 'count','clusterID']
my_df  = pd.DataFrame(columns = col_names)
my_df

,year,node,count,clusterID


In [24]:
for index, row in weighted.iterrows():
        my_df.loc[len(my_df)] = [row.year, row.source, row['count'], nodesdict[row.source.lstrip()]]
        my_df.loc[len(my_df)] = [row.year, row.target, row['count'], nodesdict[row.target.lstrip()]]
#my_df

In [25]:
for index, row in my_df.iterrows():
    #REMOVE node occurences where year is not correct - 21 rows
    if len(row.year)<>4:
        print row.year
        my_df.drop(index, inplace=True)
#my_df

cellulosic ethanol,2010
cellulosic ethanol,2010
enzymatic hydrolysis,2010
enzymatic hydrolysis,2010
ethanol,2010
ethanol,2010
hydrolysis,2010
hydrolysis,2010
anaerobic digestion,2014
anaerobic digestion,2014
biogas,2014
biogas,2014
hydrolysis,2014
hydrolysis,2014


In [51]:
#total = my_df.groupby(['year','clusterID','node']).sum()
total = my_df.groupby(['year','clusterID','node'], as_index=False, group_keys=False).sum()
total.head(30)

,year,clusterID,node,count
0,1938,5,cellulose,2
1,1938,14,sugar,2
2,1938,16,paper,2
3,1938,16,wood,2
4,1938,19,cellulosic ethanol,4
5,1938,19,ethanol,4
6,1981,11,wood chips,1
7,1981,13,gasification,2
8,1981,16,wood,1
9,1985,11,fixed bed gasifiers,2


## Write treemap structure to JSONs

In [40]:
strtotal = '{\n"name":"treemap",\n"children": [\n{'
prevyear = ''
prevcluster = ''
clID = list()
for index, row in total.iterrows():
        #print str(row.clusterID).split('\n')[0][0]
        clID = str(row.clusterID).split('\n')[0]
        #print clID
        if row.year <> prevyear:
            #new year row
            if index > 0:
                strtotal = strtotal +'\t]\n\t}\n\t]\n\t},\n\t{\n'
            strtotal = strtotal + '"name":"' + str(row.year) + '",\n'
            strtotal = strtotal + '"children": [\n{'
            if str(clID) <> prevcluster:
                #new cluster row
                strtotal = strtotal + '\t"name":"' + str(clID) + '",\n'
                strtotal = strtotal + '\t"children": [\n'
                strtotal = strtotal + '\t\t{"name":"' + row.node + '","size":' + str(row['count']) + '}'
        else:
            if str(clID) <> prevcluster:
                #new cluster row
                strtotal = strtotal +'\t]\n\t},\n\t{\n'
                strtotal = strtotal + '\t"name":"' + str(clID) + '",\n'
                strtotal = strtotal + '\t"children": [\n'
                strtotal = strtotal + '\t\t{"name":"' + row.node + '","size":' + str(row['count']) + '}'
            if clID == prevcluster:
                strtotal = strtotal + ',\n\t\t{"name":"' + row.node + '","size":' + str(row['count']) + '}'
            #strtotal = strtotal + ',\n'
            #print 'yes'
        prevyear = row.year
        prevcluster = clID
strtotal = strtotal + ']\n}\n]\n}\n]\n}'
print strtotal

{
"name":"treemap",
"children": [
{"name":"1938",
"children": [
{	"name":"5",
	"children": [
		{"name":"cellulose","size":2}	]
	},
	{
	"name":"14",
	"children": [
		{"name":"sugar","size":2}	]
	},
	{
	"name":"16",
	"children": [
		{"name":"paper","size":2},
		{"name":"wood","size":2}	]
	},
	{
	"name":"19",
	"children": [
		{"name":"cellulosic ethanol","size":4},
		{"name":"ethanol","size":4}	]
	}
	]
	},
	{
"name":"1981",
"children": [
{	"name":"11",
	"children": [
		{"name":"wood chips","size":1}	]
	},
	{
	"name":"13",
	"children": [
		{"name":"gasification","size":2}	]
	},
	{
	"name":"16",
	"children": [
		{"name":"wood","size":1}	]
	}
	]
	},
	{
"name":"1985",
"children": [
{	"name":"11",
	"children": [
		{"name":"fixed bed gasifiers","size":2},
		{"name":"wood chips","size":1}	]
	},
	{
	"name":"16",
	"children": [
		{"name":"wood","size":1}	]
	}
	]
	},
	{
"name":"1989",
"children": [
{	"name":"5",
	"children": [
		{"name":"woody biomass","size":1}	]
	},
	{
	"name":"13",
	"children": 

In [53]:
#variant 2 - break down years into separate json files
# strtotal = '{\n"name":"treemap",\n"children": [\n{'
prevyear = ''
prevcluster = ''
clID = list()
jsonsar = dict() #this will be an array of json strings for each treemap
year_counter = 0
for index, row in total.iterrows():
        #print str(row.clusterID).split('\n')[0][0]
        clID = str(row.clusterID).split('\n')[0]
        #print clID
        if row.year <> prevyear:
            #if new year, then store strtotal into new variable, write it to a file
            jsonsar[year_counter] = '{\n"name":"treemap",\n"children": [\n{' + strtotal + ']\n}\n]\n}\n]\n}'
            year_counter = year_counter + 1
            #reset strtotal
            strtotal = ''
#             if index > 0:
#                 strtotal = strtotal +'\t]\n\t}\n\t]\n\t},\n\t{\n'
            strtotal = strtotal + '"name":"' + str(row.year) + '",\n'
            strtotal = strtotal + '"children": [\n{'
            if str(clID) <> prevcluster:
                #new cluster row
                strtotal = strtotal + '\t"name":"' + str(clID) + '",\n'
                strtotal = strtotal + '\t"children": [\n'
                strtotal = strtotal + '\t\t{"name":"' + row.node + '","size":' + str(row['count']) + '}'
        else:
            if str(clID) <> prevcluster:
                #new cluster row
                strtotal = strtotal +'\t]\n\t},\n\t{\n'
                strtotal = strtotal + '\t"name":"' + str(clID) + '",\n'
                strtotal = strtotal + '\t"children": [\n'
                strtotal = strtotal + '\t\t{"name":"' + row.node + '","size":' + str(row['count']) + '}'
            if clID == prevcluster:
                strtotal = strtotal + ',\n\t\t{"name":"' + row.node + '","size":' + str(row['count']) + '}'
            #strtotal = strtotal + ',\n'
            #print 'yes'
        prevyear = row.year
        prevcluster = clID
        #strtotal = strtotal + ']\n}\n]\n}\n]\n}'
        #print strtotal
print jsonsar[1]
print '------------'
print jsonsar[2]
print '------------'
print jsonsar[8]

{
"name":"treemap",
"children": [
{"name":"1938",
"children": [
{	"name":"5",
	"children": [
		{"name":"cellulose","size":2}	]
	},
	{
	"name":"14",
	"children": [
		{"name":"sugar","size":2}	]
	},
	{
	"name":"16",
	"children": [
		{"name":"paper","size":2},
		{"name":"wood","size":2}	]
	},
	{
	"name":"19",
	"children": [
		{"name":"cellulosic ethanol","size":4},
		{"name":"ethanol","size":4}]
}
]
}
]
}
------------
{
"name":"treemap",
"children": [
{"name":"1981",
"children": [
{	"name":"11",
	"children": [
		{"name":"wood chips","size":1}	]
	},
	{
	"name":"13",
	"children": [
		{"name":"gasification","size":2}	]
	},
	{
	"name":"16",
	"children": [
		{"name":"wood","size":1}]
}
]
}
]
}
------------
{
"name":"treemap",
"children": [
{"name":"1994",
"children": [
{	"name":"4",
	"children": [
		{"name":"gasoline","size":5},
		{"name":"seed oil","size":1},
		{"name":"vegetable oil","size":1}	]
	},
	{
	"name":"5",
	"children": [
		{"name":"cellulose","size":4},
		{"name":"rapeseed","size":1

In [54]:
for key in jsonsar:
    filename = 'treemap_' + str(key) + '.json'
    with open(filename, "w") as text_file:
        text_file.write(jsonsar[key])

## Prepare data for TimeSlider

In [33]:
#count total number of node occurences per each year
nodesperyear = my_df.groupby(['year'], as_index=False, group_keys=False)['count'].sum()
nodesperyear

,year,count
0,1938,16
1,1981,4
2,1985,4
3,1989,4
4,1990,2
5,1992,8
6,1993,2
7,1994,48
8,1995,12
9,1996,72


In [37]:
#count total number of edges per each year
edgesperyear = weighted.groupby(['year'], as_index=False, group_keys=False)['count'].sum()

In [38]:
#REMOVE node occurences where year is not correct
for index, row in edgesperyear.iterrows():
    if len(row.year)<>4:
        print row.year
        edgesperyear.drop(index, inplace=True)
edgesperyear

anaerobic digestion,2014
biogas,2014
cellulosic ethanol,2010
enzymatic hydrolysis,2010
ethanol,2010
hydrolysis,2010
hydrolysis,2014


,year,count
0,1938,8
1,1981,2
2,1985,2
3,1989,2
4,1990,1
5,1992,4
6,1993,1
7,1994,24
8,1995,6
9,1996,36


In [44]:
#merge two dataframes: node occurences and edges
timeslider = pd.merge(nodesperyear, edgesperyear, on='year')
#rename resulting columns
timeslider.rename(columns={'count_x': 'nodes', 'count_y': 'edges'}, inplace=True)
timeslider

,year,nodes,edges
0,1938,16,8
1,1981,4,2
2,1985,4,2
3,1989,4,2
4,1990,2,1
5,1992,8,4
6,1993,2,1
7,1994,48,24
8,1995,12,6
9,1996,72,36


In [60]:
#find delta for each year
deltatimeslider = timeslider
prevnode = 0
prevedge = 0
deltanode = 0
deltaedges = 0
for index, row in timeslider.iterrows():
    #find delta, set this delta to deltadf
    #first, lets do nodes
    deltanode = int(row.nodes) - prevnode
    deltatimeslider.loc[deltatimeslider.index[index], 'nodes'] = deltanode
    prevnode = int(row.nodes)
    #print index, deltanode
    #same for edges
    deltaedge = int(row.edges) - prevedge
    deltatimeslider.loc[deltatimeslider.index[index], 'edges'] = deltaedge
    prevedge = int(row.edges)
    #print index, deltaedge
deltatimeslider

,year,nodes,edges
0,1938,16,8
1,1981,-44,-14
2,1985,40,6
3,1989,-12,0
4,1990,-2,-1
5,1992,10,4
6,1993,-20,-6
7,1994,64,26
8,1995,-134,-41
9,1996,178,48


In [61]:
deltatimeslider.to_csv('timeslider.csv', sep=';')

## Extract one year

In [155]:
tempyear = '2008'
oneyear = weighted.loc[weighted['year'] == tempyear]
oneyear

,edges,source,target,year,count
129,"agricultural residues,bio-oil,2008",agricultural residues,bio-oil,2008,1
160,"agricultural residues,butanol,2008",agricultural residues,butanol,2008,1
172,"agricultural residues,cellulosic ethanol,2008",agricultural residues,cellulosic ethanol,2008,1
189,"agricultural residues,ethanol,2008",agricultural residues,ethanol,2008,3
205,"agricultural residues,fermentation,2008",agricultural residues,fermentation,2008,2
232,"agricultural residues,hydrolysis,2008",agricultural residues,hydrolysis,2008,1
244,"agricultural residues,methanol,2008",agricultural residues,methanol,2008,1
254,"agricultural residues,pyrolysis,2008",agricultural residues,pyrolysis,2008,1
278,"agricultural waste,anaerobic digestion,2008",agricultural waste,anaerobic digestion,2008,1
300,"agricultural waste,biodiesel,2008",agricultural waste,biodiesel,2008,1


In [156]:
#construct list of terms that are in the edges
sourcestargets = list()
for row in oneyear.iterrows():
    #print row[1].source
    #print row[1].target
    if row[1].source not in sourcestargets:
        sourcestargets.append(row[1].source)
    if row[1].target not in sourcestargets:
        sourcestargets.append(row[1].target)
yearnodes = sourcestargets
yearnodes

[u'agricultural residues',
 u'bio-oil',
 u'butanol',
 u'cellulosic ethanol',
 u'ethanol',
 u'fermentation',
 u'hydrolysis',
 u'methanol',
 u'pyrolysis',
 u'agricultural waste',
 u'anaerobic digestion',
 u'biodiesel',
 u'biogas',
 u'fischer-tropsch',
 u'gasoline',
 u'agriculture',
 u'bioethanol',
 u'biofuels from algae',
 u'enzymatic hydrolysis',
 u'fatty acid ethyl ester',
 u'oil extraction',
 u'pellets',
 u'algae',
 u'algal oil extraction',
 u'biogasoline',
 u'renewable fuel',
 u'transesterification',
 u'animal fats',
 u'animal manure',
 u'animal waste',
 u'biobutanol',
 u'arundo donax',
 u'gasification',
 u'bagasse',
 u'banna grass',
 u'bark',
 u'fast pyrolysis',
 u'pressing',
 u'barley straw',
 u'barley',
 u'biomass gasification',
 u'syng',
 u'black liquor',
 u'biodme',
 u'blend',
 u'bioplastic',
 u'biopolymers',
 u'gas cleaning',
 u'naphtha',
 u'catalysis',
 u'cellulose',
 u'cellulosic biomass',
 u'cobs',
 u'corn cob',
 u'corn stalks',
 u'corn stover',
 u'corn',
 u'biomass combusti

In [157]:
#check if this terms are in node list, if not - remove
for term in yearnodes:
    #print term
    if term not in nodes:
        print "term: " + term + " is removed"
        yearnodes.remove(term)
yearnodes

[u'agricultural residues',
 u'bio-oil',
 u'butanol',
 u'cellulosic ethanol',
 u'ethanol',
 u'fermentation',
 u'hydrolysis',
 u'methanol',
 u'pyrolysis',
 u'agricultural waste',
 u'anaerobic digestion',
 u'biodiesel',
 u'biogas',
 u'fischer-tropsch',
 u'gasoline',
 u'agriculture',
 u'bioethanol',
 u'biofuels from algae',
 u'enzymatic hydrolysis',
 u'fatty acid ethyl ester',
 u'oil extraction',
 u'pellets',
 u'algae',
 u'algal oil extraction',
 u'biogasoline',
 u'renewable fuel',
 u'transesterification',
 u'animal fats',
 u'animal manure',
 u'animal waste',
 u'biobutanol',
 u'arundo donax',
 u'gasification',
 u'bagasse',
 u'banna grass',
 u'bark',
 u'fast pyrolysis',
 u'pressing',
 u'barley straw',
 u'barley',
 u'biomass gasification',
 u'syng',
 u'black liquor',
 u'biodme',
 u'blend',
 u'bioplastic',
 u'biopolymers',
 u'gas cleaning',
 u'naphtha',
 u'catalysis',
 u'cellulose',
 u'cellulosic biomass',
 u'cobs',
 u'corn cob',
 u'corn stalks',
 u'corn stover',
 u'corn',
 u'biomass combusti

In [158]:
#nodeindex = {0:'zero',1:'one'}
#create dictionary to map indexes to nodes
yearnodeindex = {}
for i in range(len(yearnodes)):
    yearnodeindex[i] = yearnodes[i]
yearnodeindex

{0: u'agricultural residues',
 1: u'bio-oil',
 2: u'butanol',
 3: u'cellulosic ethanol',
 4: u'ethanol',
 5: u'fermentation',
 6: u'hydrolysis',
 7: u'methanol',
 8: u'pyrolysis',
 9: u'agricultural waste',
 10: u'anaerobic digestion',
 11: u'biodiesel',
 12: u'biogas',
 13: u'fischer-tropsch',
 14: u'gasoline',
 15: u'agriculture',
 16: u'bioethanol',
 17: u'biofuels from algae',
 18: u'enzymatic hydrolysis',
 19: u'fatty acid ethyl ester',
 20: u'oil extraction',
 21: u'pellets',
 22: u'algae',
 23: u'algal oil extraction',
 24: u'biogasoline',
 25: u'renewable fuel',
 26: u'transesterification',
 27: u'animal fats',
 28: u'animal manure',
 29: u'animal waste',
 30: u'biobutanol',
 31: u'arundo donax',
 32: u'gasification',
 33: u'bagasse',
 34: u'banna grass',
 35: u'bark',
 36: u'fast pyrolysis',
 37: u'pressing',
 38: u'barley straw',
 39: u'barley',
 40: u'biomass gasification',
 41: u'syng',
 42: u'black liquor',
 43: u'biodme',
 44: u'blend',
 45: u'bioplastic',
 46: u'biopolym

In [159]:
zero_data = np.zeros(shape=(len(yearnodes),len(yearnodes)))
yearadjacency = pd.DataFrame(zero_data, index = yearnodeindex, columns=yearnodes)
yearadjacency.rename(index=yearnodeindex, inplace=True)
yearadjacency

,agricultural residues,bio-oil,butanol,cellulosic ethanol,ethanol,fermentation,hydrolysis,methanol,pyrolysis,agricultural waste,anaerobic digestion,biodiesel,biogas,fischer-tropsch,gasoline,agriculture,bioethanol,biofuels from algae,enzymatic hydrolysis,fatty acid ethyl ester,oil extraction,pellets,algae,algal oil extraction,biogasoline,renewable fuel,transesterification,animal fats,animal manure,animal waste,biobutanol,arundo donax,gasification,bagasse,banna grass,bark,fast pyrolysis,pressing,barley straw,barley,biomass gasification,syng,black liquor,biodme,blend,bioplastic,biopolymers,gas cleaning,naphtha,catalysis,cellulose,cellulosic biomass,cobs,corn cob,corn stalks,corn stover,corn,biomass combustion,renewable electricity,dedicated energy crops,digester gas,distillers grain,dry biomass,energy crops,energy grasses,food waste,forest residues,forestry,free fatty acid,glycerin,polyethylene,solvents,grain,grains,grass seed,grass straw and corn stalks,grass,hardwood,husk,jatropha,landfill gas,lignocellulosics,liquefaction,macroalgae,manure,microwave,msw,natural gas,lpg,oak,organic waste,pal waste,palm oil,palm,paper,pine,plastics,poplar,pyrolysis oil,rapeseed oil,rapeseed,refined vegetable oil,residues,rice straw,sawdust,seed oil,advanced biofuel,seeds,sewage,sorghum,soy,starch,stover,straw,sugar beet,sugar,sugarcane bagasse,sugarcane,switchgrass,syngas from gasifier,sng,syngas,vegetable oil,waste vegetable oil,waste water,waste,wheat straw,wheat,willow,wood chips,wood,woody biomass,yard trimmings,yeast
agricultural residues,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bio-oil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
butanol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cellulosic ethanol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ethanol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [160]:
for index, row in oneyear.iterrows():
    #if nodes.isin(row.source) and nodes.isin(row.target) :
    #print row['count']
    if row.source in yearnodes and row.target in yearnodes:
        yearadjacency.at[row.source,row.target] = row['count']
        yearadjacency.at[row.target,row.source] = row['count']
    else:
        print (row.source + ":" + row.target)
yearadjacency
%store yearadjacency
%store yearnodes

Stored 'yearadjacency' (DataFrame)
Stored 'yearnodes' (list)


In [161]:
%store -r nodes_clustered
nodes_clustered

,0,cluster
0,vegetable oil,4
1,woodwaste/bagasse,0
2,natural gas,14
3,various grasses,19
4,nonedible oils,17
5,animal fats,12
6,paper,16
7,ponderosa pine,19
8,grass seed,19
9,osb,14


In [168]:
filename = 'terms_by_clusters.csv'
nodes_clustered.to_csv(filename, sep=',')

In [162]:
nodes
print(len(nodes))

281


## Write the file for D3

In [163]:
strjson = '{"nodes":['
for node in yearnodes:
    print node
    #print nodeindex(node)
    #print nodes_clustered.cluster[yearnodes.index('natural gas')]
    strjson = strjson + '{"name":"' + node + '","group":' + str(nodes_clustered.cluster[nodes.index(node)]) + '},'
#remove last comma from json
strjson = strjson[:-1]
print strjson

agricultural residues
bio-oil
butanol
cellulosic ethanol
ethanol
fermentation
hydrolysis
methanol
pyrolysis
agricultural waste
anaerobic digestion
biodiesel
biogas
fischer-tropsch
gasoline
agriculture
bioethanol
biofuels from algae
enzymatic hydrolysis
fatty acid ethyl ester
oil extraction
pellets
algae
algal oil extraction
biogasoline
renewable fuel
transesterification
animal fats
animal manure
animal waste
biobutanol
arundo donax
gasification
bagasse
banna grass
bark
fast pyrolysis
pressing
barley straw
barley
biomass gasification
syng
black liquor
biodme
blend
bioplastic
biopolymers
gas cleaning
naphtha
catalysis
cellulose
cellulosic biomass
cobs
corn cob
corn stalks
corn stover
corn
biomass combustion
renewable electricity
dedicated energy crops
digester gas
distillers grain
dry biomass
energy crops
energy grasses
food waste
forest residues
forestry
free fatty acid
glycerin
polyethylene
solvents
grain
grains
grass seed
grass straw and corn stalks
grass
hardwood
husk
jatropha
landfi

In [164]:
stredges = '],"links":['
for source in yearnodes:
    #print source + "--------------"
    for target in yearnodes:
        #print target
        if yearadjacency[source][target] > 0:
            #strpair = '{"source":' + source + ',"target":' + target + ',"value":' + str(adjacency[source][target]) + '},'
            strpair = '{"source":' + str(yearnodes.index(source)) + ',"target":' + str(yearnodes.index(target)) + ',"value":' + str(yearadjacency[source][target]) + '},'
            stredges = stredges + strpair
stredges = stredges[:-1]
stredges = stredges + "]}"
print stredges

],"links":[{"source":0,"target":1,"value":1.0},{"source":0,"target":2,"value":1.0},{"source":0,"target":3,"value":1.0},{"source":0,"target":4,"value":3.0},{"source":0,"target":5,"value":2.0},{"source":0,"target":6,"value":1.0},{"source":0,"target":7,"value":1.0},{"source":0,"target":8,"value":1.0},{"source":1,"target":0,"value":1.0},{"source":1,"target":8,"value":7.0},{"source":1,"target":15,"value":1.0},{"source":1,"target":22,"value":1.0},{"source":1,"target":26,"value":1.0},{"source":1,"target":27,"value":1.0},{"source":1,"target":34,"value":1.0},{"source":1,"target":35,"value":1.0},{"source":1,"target":36,"value":2.0},{"source":1,"target":44,"value":3.0},{"source":1,"target":51,"value":1.0},{"source":1,"target":68,"value":1.0},{"source":1,"target":69,"value":1.0},{"source":1,"target":76,"value":1.0},{"source":1,"target":77,"value":1.0},{"source":1,"target":89,"value":1.0},{"source":1,"target":92,"value":1.0},{"source":1,"target":93,"value":1.0},{"source":1,"target":95,"value":2.0},

In [165]:
filename = 'clustered_' + tempyear + '.json'
with open(filename, 'w') as outfile:
    outfile.write(strjson + stredges)